**HW 5**
Answers below code!

In [275]:
# Imports and utility functions
import sympy
from sympy import nsolve, pi, E, log, S, latex, Float
from sympy.abc import x
from sympy.physics.units import hbar, nm, eV, electron_rest_mass, meter, kg, second, K, boltzmann_constant, cm, planck
from sympy.physics.units import convert_to
from sympy import sin, cos, sqrt, sinh, cosh

from sympy.plotting import plot
from sympy import integrate

from sympy.physics.units import Quantity
from IPython.display import Markdown as md

def latex_units(expr, units):
    return latex(Float(simplify(expr / units), 3)) + " " + latex(units)

def unit(expr):
    try:
        return expr.as_coeff_Mul()[1]
    except:
        return 1

def simplify(expr):
    return convert_to(expr, [kg, meter, second, K]).evalf(50)
    
def remove_units(f, x, x_units, y_units):
    f_ = convert_to(f(x * x_units), [meter, kg, second])
    f_ = f_ / y_units
    f_ = convert_to(f_, [meter, kg, second])
    return f_

def plot_with_units(f, start, end, y_units):
    x_units = unit(start + end)
    f_ = remove_units(f, x, x_units, y_units)
    return plot(f_, (x, start/x_units, end/x_units), xlabel=str(x_units), ylabel=str(y_units), show=False)

def solve_with_units(f, x_estimate, y):
    x_units = unit(x_estimate)
    y_units = unit(y)
    f_ = remove_units(f, x, x_units, y_units)
    return nsolve(f_ - y/y_units, x_estimate/x_units)

In [269]:
m_e = 1.18 * electron_rest_mass
m_h = 0.81 * electron_rest_mass

def n_i(T, E_g):
    numerator = ((m_e * m_h)**(S(3)/4) * (boltzmann_constant * T) ** (S(3)/2))
    denominator = 2 ** (S(1)/2) * pi ** (S(3)/2) * hbar ** 3
    coeff = E ** (-E_g / (2*boltzmann_constant * T))
    return simplify(numerator / denominator * coeff)

def n(N_D, N_A, T, E_g): # when n_i ~ N_D ONLY
    half_dopant_diff = (N_D - N_A) / 2
    intrinsic_conc = n_i(T, E_g)
    return simplify(half_dopant_diff + (half_dopant_diff**2 + intrinsic_conc**2)**(S(1)/2))

def p(N_D, N_A, T, E_g): # when n_i ~ N_A ONLY
    half_dopant_diff = (N_A - N_D) / 2
    intrinsic_conc = n_i(T, E_g)
    return simplify(half_dopant_diff + (half_dopant_diff**2 + intrinsic_conc**2)**(S(1)/2))

def E_i_midgap_diff(T, E_g):
    return simplify(-3/4 * boltzmann_constant * T * log(m_e/m_h))

def n_E_fE_i_diff(n_, T, E_g):
    return simplify(boltzmann_constant * T * log(n_ / n_i(T, E_g)))

def p_E_fE_i_diff(p_, T, E_g):
    return simplify(-boltzmann_constant * T * log(p_ / n_i(T, E_g)))

In [276]:
n_i_300K= n_i(300 * K, 1.12*eV)
n_i_450K= n_i(450 * K, 1.08*eV)
n_i_650K= n_i(650 * K, 1.015*eV)
result = f'''
**HW 5**

1\. **Determine the equilibrium electron and hole
concentrations in uniformly doped Si under each
of the following conditions:**

 (a) T=300K, n-doped at $N_D=10^{{15}} /cm^3$
 
 $n_i = {latex_units(n_i_300K,1/cm**3)}$
 << $N_D$, so $n_i^2 = np$.
$$\\boxed{{n=10^{{15}}, p = \\frac{{n_i^2}}{{n}} ={latex_units(n_i_300K**2/(10**15/cm**3),1/cm**3)}}}$$
 (b) T=300K, p-doped at $N_A=10^{{16}} /cm^3$

 Again, $n_i$ << $N_A$, so $n_i^2 = np$.
$$\\boxed{{p=10^{{16}}, n = \\frac{{n_i^2}}{{p}} = {latex_units(n_i_300K**2/(10**16/cm**3),1/cm**3)}}}$$
 (c) T=300K, compensated; $N_A=9×10^{{15}} /cm^3$, $N_D=10^{{16}}/cm^3$

 The net doping is $N_D - N_A = 10^{{16}} - 9×10^{{15}} = 10^{{15}}$
$$\\boxed{{n=10^{{15}}, p = \\frac{{n_i^2}}{{n}} = {latex_units(n_i_300K**2/(10**15/cm**3),1/cm**3)}}}$$
 (d) T=450K, n-doped at $N_D=10^{{14}} /cm^3$

At 450K, $n_i = {latex_units(n_i_450K,1/cm**3)}$, which is approaching $N_D$. Using the 2.29 equations:
$$\\boxed{{
    n={latex_units(n(10e14/cm**3,0,450*K, 1.08*eV),1/cm**3)},
    p={latex_units(p(10e14/cm**3,0,450*K, 1.08*eV),1/cm**3)}
}}$$
 (e) T=650K, n-doped at $N_D=10^{{14}} /cm^3$

 At 650K, $n_i = {latex_units(n_i_650K,1/cm**3)}$, which is greater than $N_D$. Again, using the 2.29 equations:
$$\\boxed{{
    n={latex_units(n(10e14/cm**3,0,650*K, 1.015*eV),1/cm**3)},
    p={latex_units(p(10e14/cm**3,0,650*K, 1.015*eV),1/cm**3)}
}}$$

2\.

I. For each of the three temperatures considered in
problem 1, where is the intrinsic Fermi level $E_i$
with respect to mid-gap?

- 300K: ${latex_units(E_i_midgap_diff(300*K, 1.12*eV),eV)}$

- 450K: ${latex_units(E_i_midgap_diff(450*K, 1.08*eV),eV)}$

- 650K: ${latex_units(E_i_midgap_diff(650*K, 1.015*eV),eV)}$

 II. For each of the five cases considered in problem
1, what is $E_F -E_i$ ?

- (a): $E_F - E_i = {latex_units(n_E_fE_i_diff(10e15/cm**3, 300*K, 1.12*eV),eV)}$

- (b): $E_F - E_i = {latex_units(p_E_fE_i_diff(10e16/cm**3, 300*K, 1.12*eV),eV)}$

- (c): $E_F - E_i = {latex_units(n_E_fE_i_diff(10e15/cm**3, 300*K, 1.12*eV),eV)}$

- (d): $E_F - E_i = {latex_units(n_E_fE_i_diff(n(10e14/cm**3,0,450*K, 1.08*eV), 450*K, 1.08*eV),eV)}$

- (e): $E_F - E_i = {latex_units(n_E_fE_i_diff(n(10e14/cm**3,0,650*K, 1.015*eV), 650*K, 1.015*eV),eV)}$

'''
display(md(result))


**HW 5**

1\. **Determine the equilibrium electron and hole
concentrations in uniformly doped Si under each
of the following conditions:**

 (a) T=300K, n-doped at $N_D=10^{15} /cm^3$
 
 $n_i = 9.49 \cdot 10^{9} \frac{1}{\text{cm}^{3}}$
 << $N_D$, so $n_i^2 = np$.
$$\boxed{n=10^{15}, p = \frac{n_i^2}{n} =9.01 \cdot 10^{4} \frac{1}{\text{cm}^{3}}}$$
 (b) T=300K, p-doped at $N_A=10^{16} /cm^3$

 Again, $n_i$ << $N_A$, so $n_i^2 = np$.
$$\boxed{p=10^{16}, n = \frac{n_i^2}{p} = 9.01 \cdot 10^{3} \frac{1}{\text{cm}^{3}}}$$
 (c) T=300K, compensated; $N_A=9×10^{15} /cm^3$, $N_D=10^{16}/cm^3$

 The net doping is $N_D - N_A = 10^{16} - 9×10^{15} = 10^{15}$
$$\boxed{n=10^{15}, p = \frac{n_i^2}{n} = 9.01 \cdot 10^{4} \frac{1}{\text{cm}^{3}}}$$
 (d) T=450K, n-doped at $N_D=10^{14} /cm^3$

At 450K, $n_i = 3.99 \cdot 10^{13} \frac{1}{\text{cm}^{3}}$, which is approaching $N_D$. Using the 2.29 equations:
$$\boxed{
    n=1.0 \cdot 10^{15} \frac{1}{\text{cm}^{3}},
    p=1.59 \cdot 10^{12} \frac{1}{\text{cm}^{3}}
}$$
 (e) T=650K, n-doped at $N_D=10^{14} /cm^3$

 At 650K, $n_i = 8.99 \cdot 10^{15} \frac{1}{\text{cm}^{3}}$, which is greater than $N_D$. Again, using the 2.29 equations:
$$\boxed{
    n=9.5 \cdot 10^{15} \frac{1}{\text{cm}^{3}},
    p=8.5 \cdot 10^{15} \frac{1}{\text{cm}^{3}}
}$$

2\.

I. For each of the three temperatures considered in
problem 1, where is the intrinsic Fermi level $E_i$
with respect to mid-gap?

- 300K: $-0.00729 \text{eV}$

- 450K: $-0.0109 \text{eV}$

- 650K: $-0.0158 \text{eV}$

 II. For each of the five cases considered in problem
1, what is $E_F -E_i$ ?

- (a): $E_F - E_i = 0.359 \text{eV}$

- (b): $E_F - E_i = -0.418 \text{eV}$

- (c): $E_F - E_i = 0.359 \text{eV}$

- (d): $E_F - E_i = 0.125 \text{eV}$

- (e): $E_F - E_i = 0.00311 \text{eV}$

